### Connecting to the service

To start working with the flight fusion service, let's start a local instance.

```sh
fusion server start
```

In [ ]:
from flight_fusion import FusionServiceClient, ClientOptions, AssetKey
from flight_fusion.ipc.v1alpha1 import SaveMode, AreaSourceReference, AreaSourceMetadata
import numpy as np
import pandas as pd
import warnings
import dagster

warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning)

# lets also define some test data to interact with the service
np.random.seed(42)
df_example = pd.DataFrame(np.random.randn(5, 3), columns=["col1", "col2", "col3"])

# and create an instance of the service client
ffc = FusionServiceClient(ClientOptions(host="localhost", port=50051))

### Discovery and metadata

In [ ]:
available_datasets = ffc.list_datasets()
available_datasets

In [7]:
fds = ffc.get_dataset_client(AssetKey(["non", "existent"]))
fds.get_metadata()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/robstar/.cache/pypoetry/virtualenvs/workspace-flight-fusion-jOezzoJ6-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4696/2944589645.py", line 2, in <cell line: 2>
    fds.get_metadata()
  File "/home/robstar/github/flight-fusion/python/flight-fusion/flight_fusion/clients/data/_dataset.py", line 83, in get_metadata
  File "pyarrow/_flight.pyx", line 1401, in pyarrow._flight.FlightClient.get_flight_info
  File "pyarrow/_flight.pyx", line 64, in pyarrow._flight.check_flight_status
pyarrow._flight.FlightCancelledError: Flight cancelled call, with message: Received RST_STREAM with error code 8

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/robstar/.cache/pypoetry/virtualenvs/workspace-flight-fusion-jOezzoJ6-py3.10/lib/python3.10/site-packages/IPython

In [ ]:
fds = ffc.get_versioned_dataset_client(available_datasets[3].asset_key)
fds.get_metadata()

In [ ]:
fds.get_metadata()

In [ ]:
import plotly.express as px

data = fds.load(columns=["col1", "col3"]).to_pandas()

In [ ]:
from app.themes import theme_light

px.scatter(data, x="col1", y="col3", template=theme_light)

### Writing and reading data

#### Dataset Client

In [ ]:
# when interacting with a single dataset, you need a dataset client
fds = ffc.get_dataset_client(AssetKey(["demo", "new_dataset"]))

In [ ]:
# write data into dataset
fds.write_into(df_example, SaveMode.SAVE_MODE_OVERWRITE)

In [ ]:
# to retrieve data, you can load the entire table ...
df = fds.load().to_pandas()

# ... or execute a query against it
df_query = fds.query("SELECT avg(col1) FROM new_dataset").to_pandas()

df.shape, df_query.shape

In [ ]:
fds.load(columns=["col2", "col3"]).to_pandas()

#### Versioned Dataset Client

In [ ]:
# getting a client for versioned or regular datasets works the same way
vds = ffc.get_versioned_dataset_client(AssetKey(["demo", "new_versioned_dataset"]))

vds.write_into(df_example, SaveMode.SAVE_MODE_OVERWRITE)

In [ ]:
df = vds.load(columns=["col1", "col3"])
df.to_pandas()

#### Querying multiple datasets

In [ ]:
fds_1 = ffc.get_dataset_client(AssetKey(["table_1"]))
fds_1.write_into(df_example)

fds_2 = ffc.get_dataset_client(AssetKey(["table_2"]))
fds_2.write_into(df_example)

ctx = ffc.get_context([("table_1", []), ("table_2", [])])
query = "SELECT * FROM table_1 UNION ALL SELECT * FROM table_2"
df = ctx.query(query).to_pandas()

df